In [1]:
import numpy as np
import numpy.linalg as LA
from sklearn.cluster import MiniBatchKMeans
from sklearn.externals import six
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import timeit
from memory_profiler import memory_usage
import math
from vlad import Vlad

In [2]:
N_COMPONENT = 2
DATA_DIR = "grasp-lift-eeg-detection/data/processed"
subjects = range(1, 2)

X =  np.concatenate([np.load("{0}/{1}/subj{2}_train_data.npy".format(DATA_DIR, N_COMPONENT, subject)) for subject in subjects])
y = np.concatenate([np.load("{0}/{1}/subj{2}_train_labels.npy".format(DATA_DIR, N_COMPONENT, subject)) for subject in subjects])

print X.shape

(1152, 32, 500)


In [3]:
print y.shape

(1152, 3)


In [4]:
vlads = Vlad(num_clusters=2048).fit(X).transform(X)

('shape of centers is ', (2048, 32))
('in transform method', (1152, 32, 500), 2048)
('X.shape is ', (1152, 500, 32))
starting for loop
('out.shape is ', (1152, 65537))


In [5]:
from sklearn.decomposition import PCA
p = PCA(n_components=0.99)

In [6]:
p.fit(vlads)

#np.save('explained_variance_ratio', p.explained_variance_ratio)

PCA(copy=True, n_components=0.99, whiten=False)

In [7]:
print p.explained_variance_ratio_.shape

(991,)


In [8]:
#print p.explained_variance_ratio_

In [9]:
ev = p.explained_variance_ratio_
print len(ev)
np.sum(ev[:800])

991


0.94624229006302807

In [10]:
fig2 = plt.figure(figsize=(20,5))
ax2 = fig2.add_subplot(111)

ax2.bar(np.arange(993), ev[:993])
plt.title('')
plt.xlabel('n_component')
plt.ylabel('Explained Variance')
plt.show()

AssertionError: incompatible sizes: argument 'height' must be length 993 or scalar

In [11]:
tmp = X.swapaxes(1,2)
tmp = tmp.reshape(tmp.shape[0]*tmp.shape[1], tmp.shape[2])

scaler = StandardScaler()
scaler.fit(tmp)
tmp = scaler.transform(tmp)


In [12]:
kmeans = MiniBatchKMeans(init='k-means++', n_clusters=6, batch_size=1000)
kmeans.fit(tmp)

centers = kmeans.cluster_centers_
clusters = kmeans.labels_

In [13]:
t_scaled = PCA(n_components=2).fit(tmp).transform(tmp)

In [14]:
clusters

array([1, 1, 1, ..., 0, 0, 0], dtype=int32)

In [ ]:
#markers = [u'o', u'o',u'0',u'1',u'2', u'3']
markers = [u'o', u'o',u'o',u'o',u'o', u'o']
colors = ['r', 'b', 'g', 'y', 'r', 'b']

fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(111)

count = 0
for t_s, cluster in zip(t_scaled,clusters):
    index = count % 500
    y_label = y[index][2]
    ax.scatter(t_s[0], t_s[1], marker=markers[int(y_label - 1)], c=colors[int(cluster)])
    count = count + 1
fig.show()




In [ ]:
t_scaled.shape

In [ ]:
y[:,2].shape

In [ ]:
clusters.shape